<center>
<h1>
    If you can't BEAT 'em, join 'em
</h1>
</center>

<center>
By the Musical Maetros
</center>


In [1]:
import numpy as np
import os
import random
import pickle
from collections import defaultdict, Counter
import mido
from mido import MidiFile, MidiTrack, Message
import glob


Make sure to download the [Maestro dataset](https://magenta.tensorflow.org/datasets/maestro) and put the directory in the same level as this notebook

## Task One

In [2]:
class SymbolicMusicGeneratorUnconditional:
    def __init__(self):
        self.trigram_model = defaultdict(Counter)
        self.tempo_transitions = defaultdict(list)
        self.note_durations = defaultdict(list)
        self.velocity_patterns = defaultdict(list)
        self.time_signatures = []
        self.tempos = []
        
    def extract_musical_events(self, midi_file_path):
        """Extract musical events from a MIDI file including timing information"""
        try:
            mid = MidiFile(midi_file_path)
            events = []
            current_tempo = 500000  # Default tempo (120 BPM)
            current_time = 0
            
            # Track active notes for duration calculation
            active_notes = {}
            
            for track in mid.tracks:
                track_time = 0
                for msg in track:
                    track_time += msg.time
                    current_time = track_time
                    
                    if msg.type == 'set_tempo':
                        current_tempo = msg.tempo
                        events.append(('tempo', msg.tempo, current_time))
                        
                    elif msg.type == 'time_signature':
                        events.append(('time_sig', f"{msg.numerator}/{msg.denominator}", current_time))
                        
                    elif msg.type == 'note_on' and msg.velocity > 0:
                        note_key = (msg.channel, msg.note)
                        active_notes[note_key] = {
                            'start_time': current_time,
                            'velocity': msg.velocity,
                            'tempo': current_tempo
                        }
                        events.append(('note_on', msg.note, current_time, msg.velocity, current_tempo))
                        
                    elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
                        note_key = (msg.channel, msg.note)
                        if note_key in active_notes:
                            note_info = active_notes[note_key]
                            duration = current_time - note_info['start_time']
                            events.append(('note_off', msg.note, current_time, duration, note_info['velocity']))
                            del active_notes[note_key]
            
            return events
            
        except Exception as e:
            print(f"Error processing {midi_file_path}: {e}")
            return []
    
    def create_symbolic_representation(self, events):
        """Convert events to symbolic representation for Markov chain"""
        symbols = []
        prev_time = 0
        
        # Filter and sort note events
        note_events = [(e[2], e) for e in events if e[0] == 'note_on']
        note_events.sort()  # Sort by time
        
        for time_pos, event in note_events:
            note = event[1]
            velocity = event[3]
            tempo = event[4]
            
            # Calculate time delta from previous note
            time_delta = max(0, time_pos - prev_time)
            prev_time = time_pos
            
            # Quantize timing to musical values (in ticks)
            if time_delta < 120:
                timing = "T1"  # Very short
            elif time_delta < 240:
                timing = "T2"  # Eighth note
            elif time_delta < 480:
                timing = "T3"  # Quarter note
            elif time_delta < 960:
                timing = "T4"  # Half note
            else:
                timing = "T5"  # Whole note or longer
            
            # Quantize velocity more musically
            if velocity < 40:
                vel_level = "pp"  # pianissimo
            elif velocity < 70:
                vel_level = "mp"  # mezzo-piano  
            elif velocity < 90:
                vel_level = "mf"  # mezzo-forte
            elif velocity < 110:
                vel_level = "f"   # forte
            else:
                vel_level = "ff"  # fortissimo
            
            # Use pitch classes and octaves for better musical coherence
            pitch_class = note % 12
            octave = note // 12
            
            # Create more musical symbol
            symbol = f"P{pitch_class}_O{octave}_{vel_level}_{timing}"
            symbols.append(symbol)
                
        return symbols
    
    def build_trigram_model(self, symbols):
        """Build trigram Markov chain from symbols"""
        for i in range(len(symbols) - 2):
            trigram_key = (symbols[i], symbols[i + 1])
            next_symbol = symbols[i + 2]
            self.trigram_model[trigram_key][next_symbol] += 1
    
    def train_on_dataset(self, dataset_path, max_files=50):
        """Train the model on MIDI files from the dataset"""
        print("Scanning for MIDI files...")
        midi_files = []
        
        # Recursively find all .mid files
        for root, dirs, files in os.walk(dataset_path):
            for file in files:
                if file.lower().endswith(('.mid', '.midi')):
                    midi_files.append(os.path.join(root, file))
        
        print(f"Found {len(midi_files)} MIDI files")
        
        # Limit the number of files to process for reasonable training time
        if len(midi_files) > max_files:
            midi_files = random.sample(midi_files, max_files)
            print(f"Processing {max_files} randomly selected files")
        
        processed_count = 0
        total_notes = 0
        octave_distribution = defaultdict(int)
        
        for midi_file in midi_files:
            print(f"Processing: {os.path.basename(midi_file)} ({processed_count + 1}/{len(midi_files)})")
            
            events = self.extract_musical_events(midi_file)
            if events:
                symbols = self.create_symbolic_representation(events)
                if len(symbols) > 10:  # Only use files with sufficient content
                    self.build_trigram_model(symbols)
                    
                    # Debug: track octave distribution
                    for symbol in symbols:
                        if symbol.startswith('P'):
                            try:
                                parts = symbol.split('_')
                                octave = int(parts[1][1:])
                                octave_distribution[octave] += 1
                                total_notes += 1
                            except:
                                pass
                    
                    # Store additional musical information
                    tempos = [e[1] for e in events if e[0] == 'tempo']
                    if tempos:
                        self.tempos.extend(tempos)
                        
                processed_count += 1
        
        print(f"Training completed on {processed_count} files")
        print(f"Learned {len(self.trigram_model)} trigram patterns")
        print(f"Total notes processed: {total_notes}")
        print("Octave distribution in training data:")
        for octave in sorted(octave_distribution.keys()):
            percentage = (octave_distribution[octave] / total_notes) * 100
            print(f"  Octave {octave}: {octave_distribution[octave]} notes ({percentage:.1f}%)")
    
    def generate_sequence(self, length=200, seed=None):
        """Generate a new sequence using the trigram model"""
        if len(self.trigram_model) == 0:
            raise ValueError("Model not trained yet!")
        
        # Start with a random trigram or use seed
        if seed and len(seed) >= 2:
            current_bigram = (seed[0], seed[1])
        else:
            current_bigram = random.choice(list(self.trigram_model.keys()))
        
        sequence = list(current_bigram)
        
        # Track musical context for better generation
        last_pitch_class = None
        last_octave = None
        octave_bias = 0  # Bias to encourage higher octaves
        
        for i in range(length - 2):
            if current_bigram in self.trigram_model:
                # Choose next symbol based on probability distribution
                next_symbols = self.trigram_model[current_bigram]
                total_count = sum(next_symbols.values())
                
                if total_count > 0:
                    # Create weighted list for better selection
                    weighted_choices = []
                    for symbol, count in next_symbols.items():
                        # Add some musical intelligence - prefer steps and skips
                        weight = count
                        if symbol.startswith('P') and last_pitch_class is not None:
                            try:
                                parts = symbol.split('_')
                                pitch_class = int(parts[0][1:])
                                octave = int(parts[1][1:])
                                
                                # Strongly favor higher octaves
                                if octave >= 5:
                                    weight = int(weight * 2.0)
                                elif octave >= 4:
                                    weight = int(weight * 1.5)
                                elif octave <= 2:
                                    weight = max(1, int(weight * 0.3))  # Heavily discourage low octaves
                                
                                # Slightly favor melodic intervals (steps and small skips)
                                interval = abs(pitch_class - last_pitch_class)
                                if interval <= 2 or interval >= 10:  # Steps (including octave wrapping)
                                    weight = int(weight * 1.2)
                                elif interval <= 4 or interval >= 8:  # Small skips
                                    weight = int(weight * 1.1)
                                
                                # Favor staying in similar octave range
                                if abs(octave - last_octave) <= 1:
                                    weight = int(weight * 1.1)
                                    
                                last_pitch_class = pitch_class
                                last_octave = octave
                            except:
                                pass
                        
                        weighted_choices.extend([symbol] * max(1, weight))
                    
                    if weighted_choices:
                        next_symbol = random.choice(weighted_choices)
                        sequence.append(next_symbol)
                        current_bigram = (current_bigram[1], next_symbol)
                    else:
                        # Fallback
                        current_bigram = random.choice(list(self.trigram_model.keys()))
                        sequence.extend(current_bigram)
                else:
                    # Fallback to random trigram
                    current_bigram = random.choice(list(self.trigram_model.keys()))
                    sequence.extend(current_bigram)
            else:
                # Start new random trigram
                current_bigram = random.choice(list(self.trigram_model.keys()))
                sequence.extend(current_bigram)
        
        return sequence
    
    def sequence_to_midi(self, sequence, output_path="generated_music.mid"):
        """Convert generated sequence back to MIDI file with proper timing"""
        mid = MidiFile(ticks_per_beat=480)
        track = MidiTrack()
        mid.tracks.append(track)
        
        # Set default tempo
        default_tempo = 500000 if not self.tempos else random.choice(self.tempos)
        track.append(mido.MetaMessage('set_tempo', tempo=int(default_tempo), time=0))
        
        # Add time signature
        track.append(mido.MetaMessage('time_signature', numerator=4, denominator=4, time=0))
        
        # Store events with absolute timing first
        events = []
        current_time = 0
        
        for i, symbol in enumerate(sequence):
            if symbol.startswith('P'):
                try:
                    parts = symbol.split('_')
                    if len(parts) >= 4:
                        pitch_class = int(parts[0][1:])
                        octave = int(parts[1][1:])
                        vel_level = parts[2]
                        timing = parts[3]
                        
                        # Constrain octave to reasonable range
                        octave = max(3, min(7, octave))
                        note = (octave * 12) + pitch_class
                        
                        # Ensure note is in valid MIDI range (0-127)
                        note = max(0, min(127, note))
                        
                        # Convert velocity level to MIDI velocity
                        velocity_map = {
                            "pp": 35, "mp": 55, "mf": 75, "f": 95, "ff": 115
                        }
                        velocity = velocity_map.get(vel_level, 70)
                        
                        # Convert timing to ticks - make them shorter for better playback
                        timing_map = {
                            "T1": 240,   # Eighth note
                            "T2": 480,   # Quarter note  
                            "T3": 720,   # Dotted quarter
                            "T4": 960,   # Half note
                            "T5": 1440   # Dotted half
                        }
                        duration = timing_map.get(timing, 480)
                        
                        # Add note on event
                        events.append(('note_on', current_time, note, velocity))
                        # Add note off event
                        events.append(('note_off', current_time + int(duration * 0.9), note, 0))
                        
                        # Advance time for next note (with some overlap allowed)
                        current_time += int(duration * 0.7)  # 70% spacing allows overlap
                        
                except (ValueError, IndexError):
                    current_time += 480  # Default quarter note spacing
            else:
                current_time += 240  # Shorter default for non-notes
        
        # Sort events by time
        events.sort(key=lambda x: x[1])
        
        # Convert to MIDI messages with delta times
        last_time = 0
        for event in events:
            event_type, abs_time, note, velocity = event
            delta_time = abs_time - last_time
            
            if event_type == 'note_on':
                track.append(mido.Message('note_on', channel=0, note=note, 
                                        velocity=velocity, time=delta_time))
            else:  # note_off
                track.append(mido.Message('note_off', channel=0, note=note, 
                                        velocity=64, time=delta_time))
            
            last_time = abs_time
        
        # Add a final rest
        track.append(mido.Message('note_off', channel=0, note=60, velocity=0, time=480))
        
        mid.save(output_path)
        print(f"Generated MIDI saved as: {output_path}")
        print(f"Total events: {len(events)}")
        print(f"Duration: ~{current_time / 480:.1f} beats")
    
    def save_model(self, filepath):
        """Save the trained model"""
        model_data = {
            'trigram_model': dict(self.trigram_model),
            'tempos': self.tempos,
            'time_signatures': self.time_signatures
        }
        with open(filepath, 'wb') as f:
            pickle.dump(model_data, f)
        print(f"Model saved to: {filepath}")
    
    def load_model(self, filepath):
        """Load a trained model"""
        with open(filepath, 'rb') as f:
            model_data = pickle.load(f)
        
        self.trigram_model = defaultdict(Counter, model_data['trigram_model'])
        self.tempos = model_data['tempos']
        self.time_signatures = model_data['time_signatures']
        print(f"Model loaded from: {filepath}") 

In [3]:
# Initialize the generator
generator = SymbolicMusicGeneratorUnconditional()

# Train on your dataset
dataset_path = "maestro-v3.0.0"
generator.train_on_dataset(dataset_path, max_files=300)  # Adjust max_files as needed

# Save the trained model
generator.save_model("music_model.pkl")

# Generate new music
print("\nGenerating new music...")
sequence = generator.generate_sequence(length=500)
generator.sequence_to_midi(sequence, "task1.mid")
print("Music generation completed!")

Scanning for MIDI files...
Found 1276 MIDI files
Processing 300 randomly selected files
Processing: MIDI-Unprocessed_079_PIANO079_MID--AUDIO-split_07-09-17_Piano-e_1-04_wav--3.midi (1/300)
Processing: MIDI-Unprocessed_16_R2_2006_01_ORIG_MID--AUDIO_16_R2_2006_03_Track03_wav.midi (2/300)
Processing: MIDI-Unprocessed_SMF_17_R1_2004_01-02_ORIG_MID--AUDIO_20_R2_2004_04_Track04_wav.midi (3/300)
Processing: ORIG-MIDI_02_7_7_13_Group__MID--AUDIO_15_R1_2013_wav--2.midi (4/300)
Processing: MIDI-Unprocessed_19_R1_2011_MID--AUDIO_R1-D7_15_Track15_wav.midi (5/300)
Processing: ORIG-MIDI_02_7_7_13_Group__MID--AUDIO_16_R1_2013_wav--2.midi (6/300)
Processing: MIDI-Unprocessed_06_R1_2009_03-07_ORIG_MID--AUDIO_06_R1_2009_06_R1_2009_05_WAV.midi (7/300)
Processing: MIDI-Unprocessed_SMF_13_01_2004_01-05_ORIG_MID--AUDIO_13_R1_2004_08_Track08_wav.midi (8/300)
Processing: MIDI-Unprocessed_23_R1_2011_MID--AUDIO_R1-D9_02_Track02_wav.midi (9/300)
Processing: MIDI-Unprocessed_12_R3_2008_01-04_ORIG_MID--AUDIO_12_R3

## Task Two

In [4]:
class SymbolicMusicGeneratorConditional:
    def __init__(self):
        self.trigram_model = defaultdict(Counter)
        self.harmony_model = defaultdict(Counter)  # For harmonization
        self.melody_model = defaultdict(Counter)   # For melody generation
        self.chord_progressions = defaultdict(Counter)  # For chord-based generation
        self.tempo_transitions = defaultdict(list)
        self.note_durations = defaultdict(list)
        self.velocity_patterns = defaultdict(list)
        self.time_signatures = []
        self.tempos = []
        
    def extract_musical_events(self, midi_file_path):
        """Extract musical events from a MIDI file including timing information"""
        try:
            mid = MidiFile(midi_file_path)
            events = []
            current_tempo = 500000  # Default tempo (120 BPM)
            current_time = 0
            
            # Track active notes for duration calculation and harmony analysis
            active_notes = {}
            simultaneous_notes = defaultdict(list)  # For harmony extraction
            
            for track in mid.tracks:
                track_time = 0
                for msg in track:
                    track_time += msg.time
                    current_time = track_time
                    
                    if msg.type == 'set_tempo':
                        current_tempo = msg.tempo
                        events.append(('tempo', msg.tempo, current_time))
                        
                    elif msg.type == 'time_signature':
                        events.append(('time_sig', f"{msg.numerator}/{msg.denominator}", current_time))
                        
                    elif msg.type == 'note_on' and msg.velocity > 0:
                        note_key = (msg.channel, msg.note)
                        active_notes[note_key] = {
                            'start_time': current_time,
                            'velocity': msg.velocity,
                            'tempo': current_tempo
                        }
                        # Group notes by time windows for harmony analysis
                        time_window = current_time // 240  # 240 ticks = eighth note window
                        simultaneous_notes[time_window].append(msg.note)
                        
                        events.append(('note_on', msg.note, current_time, msg.velocity, current_tempo))
                        
                    elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
                        note_key = (msg.channel, msg.note)
                        if note_key in active_notes:
                            note_info = active_notes[note_key]
                            duration = current_time - note_info['start_time']
                            events.append(('note_off', msg.note, current_time, duration, note_info['velocity']))
                            del active_notes[note_key]
            
            # Store harmony information for conditional generation
            self._extract_harmonies(simultaneous_notes)
            
            return events
            
        except Exception as e:
            print(f"Error processing {midi_file_path}: {e}")
            return []
    
    def _extract_harmonies(self, simultaneous_notes):
        """Extract harmony patterns from simultaneous notes"""
        for time_window, notes in simultaneous_notes.items():
            if len(notes) >= 2:  # At least 2 notes for harmony
                # Sort notes and create chord representation
                sorted_notes = sorted(set(notes))  # Remove duplicates and sort
                if len(sorted_notes) >= 2:
                    # Create melody-harmony pairs (lowest note as melody, others as harmony)
                    melody_note = sorted_notes[0] % 12  # Pitch class of lowest note
                    harmony_notes = [n % 12 for n in sorted_notes[1:]]  # Pitch classes of harmony
                    
                    # Store melody -> harmony mapping
                    harmony_key = f"M{melody_note}"
                    harmony_value = "_".join([f"H{h}" for h in sorted(harmony_notes)])
                    self.harmony_model[harmony_key][harmony_value] += 1
                    
                    # Store harmony -> melody mapping (for melody generation from chords)
                    if len(harmony_notes) >= 2:  # At least 2 harmony notes
                        chord_key = "_".join([f"C{h}" for h in sorted(harmony_notes)])
                        melody_value = f"M{melody_note}"
                        self.melody_model[chord_key][melody_value] += 1
    
    def _notes_to_chord_symbol(self, notes):
        """Convert a list of notes to a simplified chord symbol"""
        if not notes:
            return "REST"
        
        # Convert to pitch classes and sort
        pitch_classes = sorted(set([n % 12 for n in notes]))
        
        if len(pitch_classes) == 1:
            return f"SINGLE_{pitch_classes[0]}"
        elif len(pitch_classes) == 2:
            interval = (pitch_classes[1] - pitch_classes[0]) % 12
            return f"INT_{pitch_classes[0]}_{interval}"
        else:
            # Simple chord classification
            root = pitch_classes[0]
            intervals = [(pc - root) % 12 for pc in pitch_classes[1:]]
            intervals_str = "_".join(map(str, sorted(intervals)))
            return f"CHORD_{root}_{intervals_str}"
    def create_symbolic_representation(self, events):
        """Convert events to symbolic representation for Markov chain"""
        symbols = []
        prev_time = 0
        
        # Filter and sort note events
        note_events = [(e[2], e) for e in events if e[0] == 'note_on']
        note_events.sort()  # Sort by time
        
        for time_pos, event in note_events:
            note = event[1]
            velocity = event[3]
            tempo = event[4]
            
            # Calculate time delta from previous note
            time_delta = max(0, time_pos - prev_time)
            prev_time = time_pos
            
            # Quantize timing to musical values (in ticks)
            if time_delta < 120:
                timing = "T1"  # Very short
            elif time_delta < 240:
                timing = "T2"  # Eighth note
            elif time_delta < 480:
                timing = "T3"  # Quarter note
            elif time_delta < 960:
                timing = "T4"  # Half note
            else:
                timing = "T5"  # Whole note or longer
            
            # Quantize velocity more musically
            if velocity < 40:
                vel_level = "pp"  # pianissimo
            elif velocity < 70:
                vel_level = "mp"  # mezzo-piano  
            elif velocity < 90:
                vel_level = "mf"  # mezzo-forte
            elif velocity < 110:
                vel_level = "f"   # forte
            else:
                vel_level = "ff"  # fortissimo
            
            # Use pitch classes and octaves for better musical coherence
            pitch_class = note % 12
            octave = note // 12
            
            # Create more musical symbol
            symbol = f"P{pitch_class}_O{octave}_{vel_level}_{timing}"
            symbols.append(symbol)
                
        return symbols
    
    def build_trigram_model(self, symbols):
        """Build trigram Markov chain from symbols"""
        for i in range(len(symbols) - 2):
            trigram_key = (symbols[i], symbols[i + 1])
            next_symbol = symbols[i + 2]
            self.trigram_model[trigram_key][next_symbol] += 1
    
    def train_on_dataset(self, dataset_path, max_files=50):
        """Train the model on MIDI files from the dataset"""
        print("Scanning for MIDI files...")
        midi_files = []
        
        # Recursively find all .mid files
        for root, dirs, files in os.walk(dataset_path):
            for file in files:
                if file.lower().endswith(('.mid', '.midi')):
                    midi_files.append(os.path.join(root, file))
        
        print(f"Found {len(midi_files)} MIDI files")
        
        # Limit the number of files to process for reasonable training time
        if len(midi_files) > max_files:
            midi_files = random.sample(midi_files, max_files)
            print(f"Processing {max_files} randomly selected files")
        
        processed_count = 0
        total_notes = 0
        octave_distribution = defaultdict(int)
        
        for midi_file in midi_files:
            print(f"Processing: {os.path.basename(midi_file)} ({processed_count + 1}/{len(midi_files)})")
            
            events = self.extract_musical_events(midi_file)
            if events:
                symbols = self.create_symbolic_representation(events)
                if len(symbols) > 10:  # Only use files with sufficient content
                    self.build_trigram_model(symbols)
                    
                    # Debug: track octave distribution
                    for symbol in symbols:
                        if symbol.startswith('P'):
                            try:
                                parts = symbol.split('_')
                                octave = int(parts[1][1:])
                                octave_distribution[octave] += 1
                                total_notes += 1
                            except:
                                pass
                    
                    # Store additional musical information
                    tempos = [e[1] for e in events if e[0] == 'tempo']
                    if tempos:
                        self.tempos.extend(tempos)
                        
                processed_count += 1
        
        print(f"Training completed on {processed_count} files")
        print(f"Learned {len(self.trigram_model)} trigram patterns")
        print(f"Total notes processed: {total_notes}")
        print("Octave distribution in training data:")
        for octave in sorted(octave_distribution.keys()):
            percentage = (octave_distribution[octave] / total_notes) * 100
            print(f"  Octave {octave}: {octave_distribution[octave]} notes ({percentage:.1f}%)")
    
    def generate_sequence(self, length=200, seed=None):
        """Generate a new sequence using the trigram model"""
        if len(self.trigram_model) == 0:
            raise ValueError("Model not trained yet!")
        
        # Start with a random trigram or use seed
        if seed and len(seed) >= 2:
            current_bigram = (seed[0], seed[1])
        else:
            current_bigram = random.choice(list(self.trigram_model.keys()))
        
        sequence = list(current_bigram)
        
        # Track musical context for better generation
        last_pitch_class = None
        last_octave = None
        octave_bias = 0  # Bias to encourage higher octaves
        
        for i in range(length - 2):
            if current_bigram in self.trigram_model:
                # Choose next symbol based on probability distribution
                next_symbols = self.trigram_model[current_bigram]
                total_count = sum(next_symbols.values())
                
                if total_count > 0:
                    # Create weighted list for better selection
                    weighted_choices = []
                    for symbol, count in next_symbols.items():
                        # Add some musical intelligence - prefer steps and skips
                        weight = count
                        if symbol.startswith('P') and last_pitch_class is not None:
                            try:
                                parts = symbol.split('_')
                                pitch_class = int(parts[0][1:])
                                octave = int(parts[1][1:])
                                
                                # Strongly favor higher octaves
                                if octave >= 5:
                                    weight = int(weight * 2.0)
                                elif octave >= 4:
                                    weight = int(weight * 1.5)
                                elif octave <= 2:
                                    weight = max(1, int(weight * 0.3))  # Heavily discourage low octaves
                                
                                # Slightly favor melodic intervals (steps and small skips)
                                interval = abs(pitch_class - last_pitch_class)
                                if interval <= 2 or interval >= 10:  # Steps (including octave wrapping)
                                    weight = int(weight * 1.2)
                                elif interval <= 4 or interval >= 8:  # Small skips
                                    weight = int(weight * 1.1)
                                
                                # Favor staying in similar octave range
                                if abs(octave - last_octave) <= 1:
                                    weight = int(weight * 1.1)
                                    
                                last_pitch_class = pitch_class
                                last_octave = octave
                            except:
                                pass
                        
                        weighted_choices.extend([symbol] * max(1, weight))
                    
                    if weighted_choices:
                        next_symbol = random.choice(weighted_choices)
                        sequence.append(next_symbol)
                        current_bigram = (current_bigram[1], next_symbol)
                    else:
                        # Fallback
                        current_bigram = random.choice(list(self.trigram_model.keys()))
                        sequence.extend(current_bigram)
                else:
                    # Fallback to random trigram
                    current_bigram = random.choice(list(self.trigram_model.keys()))
                    sequence.extend(current_bigram)
            else:
                # Start new random trigram
                current_bigram = random.choice(list(self.trigram_model.keys()))
                sequence.extend(current_bigram)
        
        return sequence
    
    def generate_harmonization(self, melody_notes, length=None):
        """Generate harmonization for a given melody"""
        if len(self.harmony_model) == 0:
            raise ValueError("Harmony model not trained yet!")
        
        harmonized_sequence = []
        
        # If melody_notes is a list of MIDI note numbers, convert to symbols
        if isinstance(melody_notes[0], int):
            melody_symbols = []
            for note in melody_notes:
                pitch_class = note % 12
                octave = note // 12
                # Use default velocity and timing for input melody
                symbol = f"P{pitch_class}_O{octave}_mf_T3"
                melody_symbols.append(symbol)
            melody_notes = melody_symbols
        
        # Extend melody to reach target length if specified
        if length is not None:
            if len(melody_notes) < length:
                # Repeat the melody pattern to reach desired length
                extended_melody = []
                for i in range(length):
                    extended_melody.append(melody_notes[i % len(melody_notes)])
                melody_notes = extended_melody
            else:
                # If melody is already longer than target, truncate it
                melody_notes = melody_notes[:length]
        
        for melody_symbol in melody_notes:
            # Add the original melody note
            harmonized_sequence.append(melody_symbol)
            
            # Extract pitch class from melody note
            if melody_symbol.startswith('P'):
                try:
                    parts = melody_symbol.split('_')
                    pitch_class = int(parts[0][1:])
                    octave = int(parts[1][1:])
                    
                    melody_key = f"M{pitch_class}"
                    
                    if melody_key in self.harmony_model:
                        # Choose harmony based on learned patterns
                        harmony_options = self.harmony_model[melody_key]
                        total_count = sum(harmony_options.values())
                        
                        if total_count > 0:
                            # Weighted random selection
                            weighted_choices = []
                            for harmony_pattern, count in harmony_options.items():
                                weighted_choices.extend([harmony_pattern] * count)
                            
                            chosen_harmony = random.choice(weighted_choices)
                            
                            # Convert harmony pattern back to note symbols
                            harmony_notes = chosen_harmony.split('_')
                            for harmony_note in harmony_notes:
                                if harmony_note.startswith('H'):
                                    harmony_pitch = int(harmony_note[1:])
                                    # Place harmony in appropriate octave (usually higher)
                                    harmony_octave = octave if harmony_pitch > pitch_class else octave + 1
                                    harmony_octave = min(7, harmony_octave)  # Keep in range
                                    
                                    harmony_symbol = f"P{harmony_pitch}_O{harmony_octave}_mp_T3"
                                    harmonized_sequence.append(harmony_symbol)
                
                except (ValueError, IndexError):
                    pass  # Skip if parsing fails
        
        return harmonized_sequence

    def generate_melody_from_chords(self, chord_progression, length=100):
        """Generate melody that follows a chord progression"""
        if len(self.melody_model) == 0:
            raise ValueError("Melody model not trained yet!")
        
        melody_sequence = []
        
        # Convert chord progression to our internal format if needed
        if isinstance(chord_progression[0], list):
            # chord_progression is list of lists of MIDI notes
            chord_symbols = []
            for chord_notes in chord_progression:
                pitch_classes = sorted([n % 12 for n in chord_notes])
                chord_key = "_".join([f"C{pc}" for pc in pitch_classes])
                chord_symbols.append(chord_key)
            chord_progression = chord_symbols
        
        notes_per_chord = max(1, length // len(chord_progression))
        
        for chord_symbol in chord_progression:
            # Generate melody notes for this chord
            for _ in range(notes_per_chord):
                if chord_symbol in self.melody_model:
                    melody_options = self.melody_model[chord_symbol]
                    total_count = sum(melody_options.values())
                    
                    if total_count > 0:
                        # Weighted random selection
                        weighted_choices = []
                        for melody_pattern, count in melody_options.items():
                            weighted_choices.extend([melody_pattern] * count)
                        
                        chosen_melody = random.choice(weighted_choices)
                        
                        # Convert back to full symbol
                        if chosen_melody.startswith('M'):
                            pitch_class = int(chosen_melody[1:])
                            octave = random.choice([4, 5, 6])  # Melody range
                            velocity = random.choice(['mp', 'mf', 'f'])
                            timing = random.choice(['T2', 'T3', 'T4'])
                            
                            melody_symbol = f"P{pitch_class}_O{octave}_{velocity}_{timing}"
                            melody_sequence.append(melody_symbol)
                else:
                    # Fallback: generate from unconditional model
                    if len(self.trigram_model) > 0:
                        current_bigram = random.choice(list(self.trigram_model.keys()))
                        melody_sequence.extend(current_bigram)
        
        return melody_sequence[:length]
    
    def generate_style_transfer(self, input_melody, target_style_velocity="f", target_style_timing="T2"):
        """Transfer the style of an input melody (change velocity/timing patterns)"""
        styled_sequence = []
        
        # If input is MIDI notes, convert first
        if isinstance(input_melody[0], int):
            input_symbols = []
            for note in input_melody:
                pitch_class = note % 12
                octave = note // 12
                symbol = f"P{pitch_class}_O{octave}_mf_T3"
                input_symbols.append(symbol)
            input_melody = input_symbols
        
        for symbol in input_melody:
            if symbol.startswith('P'):
                try:
                    parts = symbol.split('_')
                    pitch_part = parts[0]  # P{pitch_class}
                    octave_part = parts[1]  # O{octave}
                    
                    # Apply style transfer
                    new_symbol = f"{pitch_part}_{octave_part}_{target_style_velocity}_{target_style_timing}"
                    styled_sequence.append(new_symbol)
                    
                except (ValueError, IndexError):
                    styled_sequence.append(symbol)  # Keep original if parsing fails
            else:
                styled_sequence.append(symbol)
        
        return styled_sequence
    
    def sequence_to_midi(self, sequence, output_path="generated_music.mid"):
        """Convert generated sequence back to MIDI file with proper timing"""
        mid = MidiFile(ticks_per_beat=480)
        track = MidiTrack()
        mid.tracks.append(track)
        
        # Set default tempo
        default_tempo = 500000 if not self.tempos else random.choice(self.tempos)
        track.append(mido.MetaMessage('set_tempo', tempo=int(default_tempo), time=0))
        
        # Add time signature
        track.append(mido.MetaMessage('time_signature', numerator=4, denominator=4, time=0))
        
        # Store events with absolute timing first
        events = []
        current_time = 0
        
        for i, symbol in enumerate(sequence):
            if symbol.startswith('P'):
                try:
                    parts = symbol.split('_')
                    if len(parts) >= 4:
                        pitch_class = int(parts[0][1:])
                        octave = int(parts[1][1:])
                        vel_level = parts[2]
                        timing = parts[3]
                        
                        # Constrain octave to reasonable range
                        octave = max(3, min(7, octave))
                        note = (octave * 12) + pitch_class
                        
                        # Ensure note is in valid MIDI range (0-127)
                        note = max(0, min(127, note))
                        
                        # Convert velocity level to MIDI velocity
                        velocity_map = {
                            "pp": 35, "mp": 55, "mf": 75, "f": 95, "ff": 115
                        }
                        velocity = velocity_map.get(vel_level, 70)
                        
                        # Convert timing to ticks - make them shorter for better playback
                        timing_map = {
                            "T1": 240,   # Eighth note
                            "T2": 480,   # Quarter note  
                            "T3": 720,   # Dotted quarter
                            "T4": 960,   # Half note
                            "T5": 1440   # Dotted half
                        }
                        duration = timing_map.get(timing, 480)
                        
                        # Add note on event
                        events.append(('note_on', current_time, note, velocity))
                        # Add note off event
                        events.append(('note_off', current_time + int(duration * 0.9), note, 0))
                        
                        # Advance time for next note (with some overlap allowed)
                        current_time += int(duration * 0.7)  # 70% spacing allows overlap
                        
                except (ValueError, IndexError):
                    current_time += 480  # Default quarter note spacing
            else:
                current_time += 240  # Shorter default for non-notes
        
        # Sort events by time
        events.sort(key=lambda x: x[1])
        
        # Convert to MIDI messages with delta times
        last_time = 0
        for event in events:
            event_type, abs_time, note, velocity = event
            delta_time = abs_time - last_time
            
            if event_type == 'note_on':
                track.append(mido.Message('note_on', channel=0, note=note, 
                                        velocity=velocity, time=delta_time))
            else:  # note_off
                track.append(mido.Message('note_off', channel=0, note=note, 
                                        velocity=64, time=delta_time))
            
            last_time = abs_time
        
        # Add a final rest
        track.append(mido.Message('note_off', channel=0, note=60, velocity=0, time=480))
        
        mid.save(output_path)
        print(f"Generated MIDI saved as: {output_path}")
        print(f"Total events: {len(events)}")
        print(f"Duration: ~{current_time / 480:.1f} beats")
    
    def save_model(self, filepath):
        """Save the trained model"""
        model_data = {
            'trigram_model': dict(self.trigram_model),
            'harmony_model': dict(self.harmony_model),
            'melody_model': dict(self.melody_model),
            'chord_progressions': dict(self.chord_progressions),
            'tempos': self.tempos,
            'time_signatures': self.time_signatures
        }
        with open(filepath, 'wb') as f:
            pickle.dump(model_data, f)
        print(f"Model saved to: {filepath}")
    
    def load_model(self, filepath):
        """Load a trained model"""
        with open(filepath, 'rb') as f:
            model_data = pickle.load(f)
        
        self.trigram_model = defaultdict(Counter, model_data['trigram_model'])
        self.harmony_model = defaultdict(Counter, model_data.get('harmony_model', {}))
        self.melody_model = defaultdict(Counter, model_data.get('melody_model', {}))
        self.chord_progressions = defaultdict(Counter, model_data.get('chord_progressions', {}))
        self.tempos = model_data['tempos']
        self.time_signatures = model_data['time_signatures']
        print(f"Model loaded from: {filepath}")

In [6]:
# Train and generate
generator = SymbolicMusicGeneratorConditional()
generator.train_on_dataset("maestro-v3.0.0", max_files=300)

chord_progression = [
        [60, 64, 67],  # C major (C-E-G)
        [65, 69, 72],  # F major (F-A-C)
        [67, 71, 74],  # G major (G-B-D)  
        [60, 64, 67]   # C major (C-E-G)
    ]

chord_melody = generator.generate_melody_from_chords(chord_progression, length=500)
energetic_chord = generator.generate_style_transfer(chord_melody, target_style_velocity="ff", target_style_timing="T1")
generator.sequence_to_midi(energetic_chord, "task2.mid")

Scanning for MIDI files...
Found 1276 MIDI files
Processing 300 randomly selected files
Processing: MIDI-Unprocessed_12_R2_2008_01-04_ORIG_MID--AUDIO_12_R2_2008_wav--3.midi (1/300)
Processing: MIDI-Unprocessed_043_PIANO043_MID--AUDIO-split_07-06-17_Piano-e_1-03_wav--2.midi (2/300)
Processing: MIDI-Unprocessed_09_R1_2011_MID--AUDIO_R1-D3_12_Track12_wav.midi (3/300)
Processing: MIDI-Unprocessed_060_PIANO060_MID--AUDIO-split_07-07-17_Piano-e_2-04_wav--1.midi (4/300)
Processing: MIDI-Unprocessed_09_R3_2008_01-07_ORIG_MID--AUDIO_09_R3_2008_wav--5.midi (5/300)
Processing: ORIG-MIDI_02_7_7_13_Group__MID--AUDIO_19_R1_2013_wav--1.midi (6/300)
Processing: MIDI-Unprocessed_05_R1_2011_MID--AUDIO_R1-D2_10_Track10_wav.midi (7/300)
Processing: MIDI-Unprocessed_058_PIANO058_MID--AUDIO-split_07-07-17_Piano-e_2-02_wav--2.midi (8/300)
Processing: MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.midi (9/300)
Processing: MIDI-Unprocessed_059_PIANO059_MID--AUDIO-split_07-07-17_Piano-e_2-0

In [ ]:
# HARMONY USAGE
# melody = [60, 64, 67, 65]  # C4, E4, G4, F4
# harmonized = generator.generate_harmonization(melody, 500)
# generator.sequence_to_midi(harmonized, "task2.mid")